In [1]:
categorization_path = r"%USERPROFILE%\AppData\Local\activitywatch\activitywatch\aw-server\settings.json"
# instantiate the path
import os
categorization_path = categorization_path.replace('%USERPROFILE%', os.environ['USERPROFILE'])
print(categorization_path)

C:\Users\test\AppData\Local\activitywatch\activitywatch\aw-server\settings.json


In [2]:
import json

with open(categorization_path, 'r') as file:
    classes = json.load(file)['classes']

print(classes)
classes = classes[:5]
print(classes)

# Extract a list of (name:List[str], rule:Dict)
# 
        # {
        #     "data": {
        #         "color": "#BFEF90"
        #     },
        #     "id": 8,
        #     "name": [
        #         "Explore",
        #         "PC"
        #     ],
        #     "rule": {
        #         "ignore_case": true,
        #         "regex": "\u6ce8\u518c\u8868|regedit|Taskmgr|Program Manager",
        #         "type": "regex"
        #     }
        # },
        

classes = list(map(lambda x: (x["name"], x["rule"]), classes))
print(classes)

[{'data': {'color': '#7FDFFF', 'score': 10}, 'id': 0, 'name': ['Work'], 'rule': {'type': 'none'}}, {'data': {'color': '#AFDFFF'}, 'id': 1, 'name': ['Work', 'Programming'], 'rule': {'ignore_case': True, 'regex': 'csdn|ipynb|python|powershell|terminal|vscode|cursor|GitHub|Stack Overflow|BitBucket|Gitlab|vim|Spyder|kate|Ghidra|Scite', 'type': 'regex'}}, {'data': {'color': '#8FBFFF'}, 'id': 2, 'name': ['Work', 'PaperWork'], 'rule': {'type': 'none'}}, {'data': {'color': '#009CE0'}, 'id': 3, 'name': ['Work', 'Research'], 'rule': {'ignore_case': True, 'regex': 'research|scholar|review|arxiv|zotero|overleaf|latex|paper|科研|论文|研究|NIPS|ICML|ICRA|AAMAS|CVPR|Conference', 'type': 'regex'}}, {'data': {'color': '#AFEFA0'}, 'id': 4, 'name': ['Explore'], 'rule': {'type': 'none'}}, {'data': {'color': '#8FEF80'}, 'id': 5, 'name': ['Explore', 'IM'], 'rule': {'regex': 'TIM|QQ|Gmail|Thunderbird|mutt|alpine|outlook|qq.com|微信|weixin|wechat|qq|tim|Messenger|Telegram|Signal|WhatsApp|Rambox|Slack|Riot|Element|Dis

In [3]:
import aw_client 
from aw_client import queries
import socket
from datetime import datetime, time, timedelta
td1d = timedelta(days=1)
day_offset = timedelta(hours=4)

hostname = socket.gethostname()
now = datetime.now().astimezone()
today = (datetime.combine(now.date(), time()) + day_offset).astimezone()
print(today)

timeperiods = [(today - i * td1d, today - (i - 1) * td1d) for i in range(1)]
timeperiods.reverse()


aw = aw_client.ActivityWatchClient()

canonicalQuery = queries.canonicalEvents(
    queries.DesktopQueryParams(
        bid_window=f"aw-watcher-window_{hostname}",
        bid_afk=f"aw-watcher-afk_{hostname}",
        classes=classes,
    )
)

query = f"""
{canonicalQuery}
duration = sum_durations(events);
RETURN = {{"events": events, "duration": duration}};
"""


res = aw.query(query, timeperiods)


2024-09-29 04:00:00+08:00


In [4]:


from toolz.curried import reduceby
from aw_core import Event
from aw_transform import flood
from typing import List


smallevent = (res[0]['events'])
smallevent = flood(map(lambda x: Event(**x),smallevent), 0) ##type:ignore

cats= reduceby(lambda x: x['data']['$category'].__repr__(), 
            lambda acc,x: acc+x['duration'].total_seconds(), 
            smallevent,
            0
            )

print(cats)


{"['Uncategorized']": 7581.233000000004, "['Work', 'Research']": 700.321, "['Work', 'Programming']": 10403.224000000011}


In [5]:

dur = res[0]['duration']
print(res[0]['duration'])
print(cats.values())
# print(sum(map(lambda x: x.total_seconds(),cats.values())))

catratio = {k:v/dur for k,v in cats.items()}
print(catratio)

18684.778
dict_values([7581.233000000004, 700.321, 10403.224000000011])
{"['Uncategorized']": 0.40574380921196945, "['Work', 'Research']": 0.03748083065263072, "['Work', 'Programming']": 0.5567753601354007}


In [6]:
from treetype import TreeType
    
tree = TreeType.tree_expand(catratio)
print((tree['Work']).sum())
print(tree)

0.5942561907880315
{'Uncategorized': {'__root__': 0.40574380921196945}, 'Work': {'Research': {'__root__': 0.03748083065263072}, 'Programming': {'__root__': 0.5567753601354007}}}


In [15]:
cons =[
        {
            "term": ["Work"],
            "op": "ge",
            "value": 0.5
        },
        {
            "term": ["Work"],
            "op": "ge",
            "value": 0.6
        }
    ]
# from functools import reduce
from toolz.curried import curry
from fn import F
from toolz.curried import reduce,juxt
import operator

get_term = (F()
            << TreeType.sum
            << reduce(lambda acc,y:acc[y])
            )

print(get_term(['Work'],tree))

# res = ops(get_term())
def check_a_constraint(constraint,tree):
    return getattr(operator,constraint['op']) (tree.get_term(constraint['term']).sum(),constraint['value'])
    

print(all(juxt(map(curry(check_a_constraint),cons))(tree)))


0.5942561907880315
False
